In [1]:
import os
import numpy as np
import pandas as pd
import scipy.stats
from sklearn.linear_model import LinearRegression as lr
import datetime as dt
%matplotlib inline
import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')


In [ ]:
def risk_factor(concentration,contaminant):
    cont_factors = {"benzene":0.1} #non-carcinogens to be filled in 1/ref_dose
    
    weight = 70 #kg
    ir = .003 * 25 * 3600 #m^3/breath * breath/min * min/day = m^3/day
    
    #freq is number days/year exposure happens; duration is #years covered in analysis
    freq = 180 # days/year
    duration = 3 # years
    avging_time = duration * 365 #days, ultimately
    ef = (freq * duration)/avging_time
    
    add = (concentration * ir * ef)/weight
    risk = add*cont_factors[contaminant]
    return risk

In [4]:
dfmain = pd.read_csv('Masterlist.csv',header = 0,parse_dates = ['Date_Time_NYC'])

In [6]:
np.mean(dfmain['PM2.5_calib'].loc[dfmain['Loc_code'].isin(['PU','T'])])

105.44145079265178

In [9]:
dfmain['Location'].value_counts()

Apartment (Chelsea)               2704
Apartment (UWS)                   1587
ABOVEGROUND                        194
116th St - Columbia University      89
Lexington Ave - 63rd St             54
                                  ... 
Canal St - Holland Tunnel            1
Jefferson St                         1
Metropolitan Ave                     1
Wilson Ave                           1
ELEVATOR                             1
Name: Location, Length: 132, dtype: int64

In [23]:
dfmain['PM2.5_calib'].loc[dfmain['Location'] == '168th St'].value_counts()

448.189214    1
543.647696    1
465.651131    1
113.402107    1
Name: PM2.5_calib, dtype: int64

In [37]:
locations = dfmain['Location'].unique()
loc_means = {}
for l in locations:
    loc_means[l] = np.mean(dfmain['PM2.5_calib'].loc[dfmain['Location'] == l])
# sorted(loc_means, key=loc_means.get, reverse = True)
# 'Lower East Side - 2nd Ave', '168th St'
loc_means['Lexington Ave - 63rd St']

113.18727153753042

In [35]:
np.mean(dfmain['PM2.5_calib'].loc[dfmain['Location'] == '116th St - Columbia University'])

61.85754009254726

In [ ]:
# Sample trip for exposure analysis: 

start = dt.datetime.strptime('2021-03-24 16:19', '%Y-%m-%d %H:%M')
end = dt.datetime.strptime('2021-03-24 17:49', '%Y-%m-%d %H:%M')

daytrip = dfmain.loc[(dfmain['Date_Time_NYC'] >= start) & (dfmain['Date_Time_NYC'] <= end)]
daytrip

In [ ]:
fig, ax = plt.subplots()
ax.plot(daytrip['Date_Time_NYC'],daytrip['PM2.5_calib'])

In [ ]:
np.mean(daytrip['PM2.5_calib'].loc[daytrip['Loc_code'].isin(['I','O'])])

In [ ]:
codes = daytrip['Loc_code'].unique()
code_means = {}
for s in codes:
    code_means[s] = np.mean(daytrip['PM2.5_calib'].loc[daytrip['Loc_code'] == s])

code_errs = {}
for t in codes:
    code_errs[t] = np.std(daytrip['PM2.5_calib'].loc[daytrip['Loc_code'] == t])/len(
        daytrip['PM2.5_calib'].loc[daytrip['Loc_code'] == t])

for c in codes:
    print('%s:')%(c)

In [5]:
codes = dfmain['Loc_code'].unique()
code_means = {}
for s in codes:
    code_means[s] = np.mean(dfmain['PM2.5_calib'].loc[dfmain['Loc_code'] == s])

code_errs = {}
for t in codes:
    code_errs[t] = np.std(dfmain['PM2.5_calib'].loc[dfmain['Loc_code'] == t])/len(
        dfmain['PM2.5_calib'].loc[dfmain['Loc_code'] == t])

In [ ]:
# Order: PU, T, PA, I, O, E, F
# Exposure scalars calculated as weighted averages of their representative
#     location codes
hippie_exp = code_means['I']*(4337/4630)+code_means['O']*(293/4630)
mole_exp = code_means['PU']*(274/603)+code_means['T']*(287/603) + code_means['PA']*(42/603)

# Example exposure profiles based on time spent underground
commuter = hippie_exp*(23/24)+mole_exp*(1/24)
worker = hippie_exp*(16/24)+mole_exp*(8/24)

print(hippie_exp,mole_exp,commuter,worker)

In [ ]:
test = dfmain['Loc_code'].value_counts()
test

In [ ]:
((12-hippie_exp)/(mole_exp-hippie_exp) * 24)*60-60

In [ ]:
# Quick function, evaluates exposure based on hours input for commute
commute = lambda t: mole_exp*(t/24) + hippie_exp*((24-t)/24)

commute(8)/12